<a href="https://colab.research.google.com/github/wenzhenghe7/web_scraping/blob/main/b%E7%AB%99%E6%8E%92%E8%A1%8C%E6%A6%9C%E6%8A%93%E5%8F%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bilibili_api
from bs4 import BeautifulSoup
import requests
from openpyxl import Workbook
from bilibili_api import user
import time
import os
import sys
import smtplib
from email import encoders
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart

In [ ]:
def get_top_100(n_limit = 100, if_get_up_detail = True, follower_limit = 200000, first_video_gap = 90):
    source = requests.get("https://www.bilibili.com/v/popular/rank/all").text
    soup = BeautifulSoup(source, 'html.parser')
    ranking_info = []
    for i,item in enumerate(soup.find_all('div', {'class': 'info'})):
        if i+1 > n_limit:
            break
        temp_1 = item.find_all('span', {'class' : 'data-box'})
        temp_2 = item.find_all('a')
        rank = i+1
        link = item.a['href'][2:]
        title = item.a.text
        play = temp_1[0].text.strip()
        view = temp_1[1].text.strip()
        author = temp_1[2].text.strip()
        pts = item.find('div', {'class' : 'pts'}).div.text
        author_page = temp_2[1]['href'][2:]
        author_id = author_page[19:]
        output_info = [rank,link,title,play, view, author, pts, author_page, author_id]
        if if_get_up_detail:
            auther_detail = user.get_relation_info(int(author_id))
        if auther_detail['follower'] <= follower_limit:
            video_detail = user.get_videos(int(author_id))
            first_po_time = min([i['created'] for i in video_detail])
            if (time.time()-first_po_time)/(24*60*60) <= first_video_gap:
                first_po_time_format = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(int(first_po_time)))
                output_info.append(auther_detail['follower'])
                output_info.append(first_po_time_format)
                ranking_info.append(output_info)
            else:
                continue
        else:
            continue
        print('排名第{}位符合要求'.format(rank))
    return ranking_info
def export_video_list_excel(video_list):
    wb = Workbook(write_only=True)
    ws = []
    ws.append(wb.create_sheet(title='bili_top_100'))
    ws[0].append(['排名', '视频链接', '标题', '播放数', '弹幕数', 'up主', '综合得分','up主链接','up主id','粉丝数','首视频日期'])
    for bl in video_list:
      ws[0].append(bl)
    save_path = 'bilibili_top_100_'
    # save_path += time.strftime(time.ctime())
    save_path += time.strftime("%Y-%m-%d", time.localtime())
    save_path += '.xlsx'
    wb.save(save_path)
    print('xlsx文档生成完毕')
    return save_path
def send_mail(sub,content,mail_to,fileList):
    mail_host='smtp.163.com'
    mail_user='autowebscrapper@163.com'
    mail_pass='GTRVVSQEADSVACSG'
    mail_to_list=mail_to.split(',')
 
    msg = MIMEMultipart()
    #msg=MIMEText(content,_charset='utf-8')
    msg['Subject']=sub
    msg['From']=mail_user
    msg['To']=";".join(mail_to_list)
    msg.attach(MIMEText(content,_charset='utf-8'))
 
    files=fileList.split(',')
    for f in files:
        part = MIMEBase('application', 'octet-stream') #'octet-stream': binary data
        part.set_payload(open(f, 'rb').read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment; filename="%s"' % os.path.basename(f).encode('utf-8'))
        msg.attach(part)
 
    try:
        s=smtplib.SMTP_SSL()
        s.connect(mail_host,port = 994)
        s.login(mail_user,mail_pass)
        s.sendmail(mail_user,mail_to_list,msg.as_string())
        s.close()
        return '发送成功'
    except Exception as e:
        print(str(e))
        return '发送失败'
 


In [ ]:
result = get_top_100()

排名 46 符合要求
排名 64 符合要求


In [ ]:
save_path = export_video_list_excel(result)

xlsx文档生成完毕


In [ ]:
send_mail(save_path, '请下载附件','elninozou@me.com',save_path)

'发送成功'